In [19]:
import pecos as pc
import numpy as np
import itertools

In [3]:
pc.__version__

'0.2.dev'

**Task**: Verify fault-tolerance of the $|0_L\rangle$-preparation circuit with flag qubit.
* The Stabilizer generators of the Steane code are pure (don't mix X/Z): <IIIZZZZ,ZIZIZIZ,IZZIIZZ> + <IIIXXXX,XIXIXIX,IXXIIXX>.
* This circuit **prepares** the state $|0_L\rangle$ of the Steane code (it neither measures syndromes nor corrects any errors).
* The $|0_L\rangle$ and $|1_L\rangle$ states are the codewords, thus they are +1 eigenstates of all stabilizers (X- **and** Z-stabilizers)
* $|0_L\rangle=|0000000\rangle+|1010101\rangle+|0110011\rangle+|1100110\rangle+|0001111\rangle+|1011010\rangle+|0111100\rangle+|1101001\rangle$ and $|1_L\rangle=X_L|0_L\rangle=|1111111\rangle+|0101010\rangle+|1001100\rangle+|0011001\rangle+|1110000\rangle+|0100101\rangle+|1000011\rangle+|0010110\rangle$ are unique linear combinations of the 16 codewords of the Hamming $[7,4,3]$ code which become the two codwords of the Steane code by construction via dual code.
* **Note** that both $|0_L\rangle$ and $|1_L\rangle$ are +1 eigenstates of the stabilizers, but they correspond to $\pm1$-eigenstates of the logical operator $Z_L$.
* This circuit guarantees that for any possible single- or 2-qubit fault the output state will be the $|0_L\rangle$ state with max. 1 error (X,Y,Z) on 1 qubit, which a subsequent round of error-correction (not implemented) is capable to resolve.
* **Note**: *Faults* arise in the circuit and might cause *errors* in the output state by propagation. We are considering 1- and 2-qubit faults in the circuit and know the Steane code is capable to correct a 1-qubit error (in output).
* For a physical state which is **more than** 1 error away from the closest codeword (stabilizer bitstring), the flag qubit will be on, telling us to discard the state, as we cannot correct it.
    * In this case, the state would be interpreted as a $|1_L\rangle$ state with a single-qubit error, which will subsequently be corrected to $|1_L\rangle$, for example $|\psi\rangle=|\textbf{1}0\textbf{1}0000\rangle+|\textbf{0}0\textbf{0}0101\rangle+|\textbf{1}1\textbf{0}0011\rangle+|\textbf{0}1\textbf{1}0110\rangle+|\textbf{1}0\textbf{1}1111\rangle+|\textbf{0}0\textbf{0}1010\rangle+|\textbf{1}1\textbf{0}1100\rangle+|\textbf{0}1\textbf{1}1001\rangle$, which is closer to the $|1_L\rangle$ state than to the $|0_L\rangle$ state. 
    * It is more advantageous to think about how the stabilizer generators (i.e. the symmetries of the code space) change due to errors and not how the state changes. Thus, for the $X_1X_3$ error above the generators change like $<XIXZZZ,-YIYIZIZ,iXZYIIZZ,\textbf{XIXXXXX},IIIIXIX,XXIIIXX>$. Note the closeness of the fourth stabilizer generator to the logical X-operator $X_L$ and a distance of 2 to the origin generator $IIIXXXX$.
    * Conversely, we can check if the inverse of an output stabilizer generators has a distance less or equal than 1 to one of the original generators.

In [18]:
# Build circuit
qc = pc.circuits.QuantumCircuit()
qc.append('init |0>',{0,4,5,6})
qc.append('init |+>',{1,2,3})
qc.append('CNOT',{(1,0),(3,5)})
qc.append('CNOT',{(2,6)})
qc.append('CNOT',{(1,4)})
qc.append('CNOT',{(2,0),(3,6)})
qc.append('CNOT',{(1,5)})
qc.append('CNOT',{(6,4)})
qc.append('CNOT',{(0,7)})
qc.append('CNOT',{(5,7)})
qc.append('CNOT',{(6,7)})
qc.append('measure Z', {0,1,2,3,4,5,6})

# All possible 1-qb and 2-qb faults
faults_1 = ['X', 'Y', 'Z']
faults_2 = list(itertools.product(faults_1 + ["I"], repeat=2)) # variation with repetition
faults_2.remove(('I','I')) # remove II as it is not a fault

def get_fcirc(tick, circuit_setup):
    fc = pc.error_gens.ErrorCircuits()
    qc = pc.circuits.QuantumCircuit(circuit_setup=circuit_setup)
    fc.add_circuits(time=tick, after_faults=qc)
    return fc

# Generate all possible faults at all possible positions in the circuit
gates_1 = [(i, locs) for i,(_,locs,_) in enumerate(qc.items()) if all(type(l) is int for l in locs)]
gates_2 = [(i, locs) for i,(_,locs,_) in enumerate(qc.items()) if all(type(l) is tuple for l in locs)]
f1_circs = [get_fcirc(t,[{f:{l}}]) for (t,ls) in gates_1 for l in ls for f in faults_1]
f2_circs = [get_fcirc(tick,[{fault:{loc}} for fault, loc in zip(ftup,ltup) if fault != 'I']) for tick,locs in gates_2 for ltup in locs for ftup in faults_2]

# CSS Stabilizer generators are pure (no X-Z-mixing)
# thus can be represented in simple binary form (111 instead of 111|000)
k1 = 0b0001111 # IIIPPPP
k2 = 0b1010101 # PIPIPIP
k3 = 0b0110011 # IPPIIPP
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3] # stabs: powerset of generators, 0: identity

hamming2 = lambda a,b: bin(a^b).count('1') # Hamming dist: # of bit pairs with odd parity

# Create bitstring from list of indices for '1's left to right (MSB) 
# Need MSB due to naming convention of Pauli strings, expl. IIX = X3 -> 001
def bin_from_ids(indices, bitlen=7):
    bit_string = 0
    for index in indices:
        bit_string |= 2**(bitlen-index-1) # 7-bit number 2**0..6
    return bit_string

n_qbs = 7 # no errors on flag qubit
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
for f in f1_circs + f2_circs:
    
    # Propagate errors through circuit & measure
    state = pc.simulators.PauliFaultProp(n_qbs)
    msmt, _ = circ_runner.run(state, qc, error_circuits=f) # updates 'state' to U|psi> & (non-destructive) msmt
    error = state.faults # error caused by propagated fault(s).
    
    # Check if any operator has a distance of 1 or 0 to logical X
    msmt_str = bin_from_ids(list(*msmt.values()))
    x_L_err = 1 if min([hamming2(msmt_str^0b1111111, stab) for stab in stabs]) <= 1 else 0
    if x_L_err: print(error)

{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': {2, 3}}
{'X': {0, 2, 7}, 'Y': set(), 'Z': {3, 6}}
{'X': {0, 2, 7}, 'Y': set(), 'Z': set()}
{'X': {0, 7}, 'Y': {2}, 'Z': {3, 6}}
{'X': {0, 7}, 'Y': {2}, 'Z': set()}
{'X': {4, 6, 7}, 'Y': set(), 'Z': {2}}
{'X': {4, 7}, 'Y': {6}, 'Z': {2, 3}}
{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': {3}}
{'X': {1, 5, 7}, 'Y': set(), 'Z': {4, 6}}
{'X': {1, 5, 7}, 'Y': set(), 'Z': set()}
{'X': {5, 7}, 'Y': {1}, 'Z': {4, 6}}
{'X': {5, 7}, 'Y': {1}, 'Z': set()}
{'X': {0, 2, 7}, 'Y': set(), 'Z': set()}
{'X': {2, 7}, 'Y': {0}, 'Z': set()}
{'X': {0, 7}, 'Y': {2}, 'Z': set()}
{'X': {7}, 'Y': {0, 2}, 'Z': set()}
{'X': {4, 6, 7}, 'Y': set(), 'Z': {3}}
{'X': {4, 7}, 'Y': {6}, 'Z': {3}}
{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': set()}
{'X': {1, 5, 7}, 'Y': set(), 'Z': set()}
{'X': {1, 7}, 'Y': {5}, 'Z': set()}
{'X': {5, 7}, 'Y': {1}, 'Z': set()}
{'X': {7}, 'Y': {1, 5}, 'Z': set()}
{'X': {4, 6